# Getting Started with CREST
CREST is a hybrid modelling DSL (domain-specific language) that focuses on the flow of resources within cyber-physical systems (CPS).

CREST is implemented in the Python programming language as the `crestdsl` internal DSL and shipped as Python package.  
`crestdsl`'s source code is hosted on GitHub https://github.com/stklik/CREST/


## This Notebook
The purpose of this notebook is to provide a small showcase of modelling with `crestdsl`.
The system to be modelled is a growing lamp that produces light and heat, if the lamp is turned on and electricity is provided.

## How to use this Jupyter notebook:

Select a code-cell (such as the one directly below) and click the `Run` button in the menu bar above to execute it. (Alternatively, you can use the keyboard combination `Ctrl+Enter`.)

**Output:** will be shown directly underneath the cell, if there is any.

To **run all cells**, you can iteratively execute individual cells, or execute all at once via the menu item `Cell` -> `Run all`  

Remember, that the order in which you execute cells is important, not the placement of a cell within the notebook.

For a more profound introduction, go and visit the [Project Jupyter](http://jupyter.org/) website. 

In [ ]:
print("Try executing this cell, so you ge a feeling for it.")

2 + 2  # this should print "Out[X]: 4" directly underneath (X will be an index)

# Defining a `crestdsl` Model

## Import `crestdsl`

In order to use `crestdsl`, you have to import it.
Initially, we will create work towards creating a system model, so let's import the `model` subpackage.

In [ ]:
import crestdsl.model as crest

## Define Resources 
First, it is necessary to define the resource types that will be used in the application.
In CREST and `crestdsl`, resources are combinatinos of resource names and their value domains.

Value domains can be infinite, such as Real and Integers or discrete such as `["on", "off"]`, as shown for the switch.

In [ ]:
electricity = crest.Resource("Watt", crest.REAL)
switch = crest.Resource("switch", ["on", "off"])
light = crest.Resource("Lumen", crest.INTEGER)
counter = crest.Resource("Count", crest.INTEGER)
time = crest.Resource("minutes", crest.REAL)
celsius = crest.Resource("Celsius", crest.REAL)
fahrenheit = crest.Resource("Fahrenheit", crest.REAL)

## Our First Entity
In CREST any system or component is modelled as Entity.
Entities can be composed hierachically  (as we will see later).

To model an entity, we define a Python class that inherits from `crest.Entity`.

Entities can define 
 - `Input`, `Output` and `Local` ports (variables),
 - `State` objects and a `current` state
 - `Transition`s between states
 - `Influence`s between ports (to express value dependencies between ports)
 - `Update`s that are continuously executed and write values to a port
 - and `Action`s, which allow the modelling of discrete changes during transition firings.
 
 
 Below, we define the `LightElement` entity, which models the component that is responsible for producing light from electricity. It defines one input and one output port.

In [ ]:
class LightElement(crest.Entity):
    """This is a definition of a new Entity type. It derives from CREST's Entity base class."""
    
    """we define ports - each has a resource and an initial value"""
    electricity_in = crest.Input(resource=electricity, value=0)
    light_out = crest.Output(resource=light, value=0)
    
    """automaton states - don't forget to specify one as the current state"""
    on = crest.State()
    off = current = crest.State()
    
    """transitions and guards (as lambdas)"""
    off_to_on = crest.Transition(source=off, target=on, guard=(lambda self: self.electricity_in.value >= 100))
    on_to_off = crest.Transition(source=on, target=off, guard=(lambda self: self.electricity_in.value < 100))
    
    """
    update functions. They are related to a state, define the port to be updated and return the port's new value
    Remember that updates need two parameters: self and dt.
    """
    @crest.update(state=on, target=light_out)
    def set_light_on(self, dt=0):
        return 800

    @crest.update(state=off, target=light_out)
    def set_light_off(self, dt=0):
        return 0

## Visualising Entities
By default, CREST is a graphical language. Therefore it only makes sense to implement a graphical visualisation of `crestdsl` systems.

One of the plotting engines is defined in the `crestdsl.ui.elk` module.
The code below produces an interactive HTML output. 

You can easily interact with the model to explore it:

- Move objects around if the automatic layout does not provide an sufficiently good layout.
- Select ports and states to see their outgoing arcs (blue) and incoming arcs (red).
- Hover over transitions, influences and actions to display their name and short summary.
- Double click on transitions, influences and actions you will see their source code.
- There is a *hot corner* on the top left of each entity. You can double-click it to collapse the entity. This feature is useful for CREST diagrams with many entities. *Unfortunately a software issue prevents the expand/collapse icon not to be displayed. It still works though (notice your cursor changing to a pointer)*

 
**GO AHEAD AND TRY IT**

In [ ]:
# import the plotting libraries that can visualise the CREST systems
from crestdsl.ui import elk

elk.plot(LightElement())

## Define Another Entity (The HeatElement)
It's time to model the heating component of our growing lamp.
Its functionality is simple: if the `switch_in` input is `on`, 1% of the electricity is converted to addtional heat under the lamp.
Thus, for example, by providing 100 Watt, the temperature underneath the lamp grows by 1 degree centigrade.

In [ ]:
class HeatElement(crest.Entity):
    """ Ports """
    electricity_in = crest.Input(resource=electricity, value=0)
    switch_in = crest.Input(resource=switch, value="off")  # the heatelement has its own switch
    heat_out = crest.Output(resource=celsius, value=0)      # and produces a celsius value (i.e. the temperature increase underneath the lamp)
    
    """ Automaton (States) """
    state = current = crest.State() # the only state of this entity
    
    """Update"""
    @crest.update(state=state, target=heat_out)
    def heat_output(self, dt):
        # When the lamp is on, then we convert electricity to temperature at a rate of 100Watt = 1Celsius
        if self.switch_in.value == "on":
            return self.electricity_in.value / 100
        else:
            return 0

# show us what it looks like
elk.plot(HeatElement())

## Adder - A Logical Entity

CREST does not specify a special connector type that defines what is happening for multiple incoming influence, etc. Instead standard entities are used to define add, minimum and maximum calculation which is then written to the actual target port using an influence.

We call such entities *logical*, since they don't have a real-world counterpart.

In [ ]:
# a logical entity can inherit from LogicalEntity, 
# to emphasize that it does not relate to the real world
class Adder(crest.LogicalEntity):
    heat_in = crest.Input(resource=celsius, value=0)
    room_temp_in = crest.Input(resource=celsius, value=22)
    temperature_out = crest.Output(resource=celsius, value=22)
    
    state = current = crest.State()
    @crest.update(state=state, target=temperature_out)
    def add(self, dt):
        return self.heat_in.value + self.room_temp_in.value
    
elk.plot(Adder())  # try adding the display option 'show_update_ports=True' and see what happens!

## Put it all together - Create the `GrowLamp`
Finally, we create the entire `GrowLamp` entity based on the components we already created.
We define subentities in a similar way to all other definitions - as class variables.

Additionally, we use influences to connect the ports to each other.

In [ ]:
class GrowLamp(crest.Entity):
    
    """ - - - - - - - PORTS - - - - - - - - - - """
    electricity_in = crest.Input(resource=electricity, value=0)
    switch_in = crest.Input(resource=switch, value="off")
    heat_switch_in = crest.Input(resource=switch, value="on")
    room_temperature_in = crest.Input(resource=fahrenheit, value=71.6)
    
    light_out = crest.Output(resource=light, value=3.1415*1000) # note that these are bogus values for now
    temperature_out = crest.Output(resource=celsius, value=4242424242) # yes, nonsense..., they are updated when simulated
    
    on_time = crest.Local(resource=time, value=0)
    on_count = crest.Local(resource=counter, value=0)
    
    """ - - - - - - - SUBENTITIES - - - - - - - - - - """
    lightelement = LightElement()
    heatelement = HeatElement()
    adder = Adder()
    
    
    """ - - - - - - - INFLUENCES - - - - - - - - - - """
    """
    Influences specify a source port and a target port. 
    They are always executed, independent of the automaton's state.
    Since they are called directly with the source-port's value, a self-parameter is not necessary.
    """
    @crest.influence(source=room_temperature_in, target=adder.room_temp_in)
    def celsius_to_fahrenheit(value):
        return (value - 32) * 5 / 9
    
    # we can also define updates and influences with lambda functions... 
    heat_to_add = crest.Influence(source=heatelement.heat_out, target=adder.heat_in, function=(lambda val: val))
    
    # if the lambda function doesn't do anything (like the one above) we can omit it entirely...
    add_to_temp           = crest.Influence(source=adder.temperature_out, target=temperature_out)
    light_to_light        = crest.Influence(source=lightelement.light_out, target=light_out)
    heat_switch_influence = crest.Influence(source=heat_switch_in, target=heatelement.switch_in)
    
    
    """ - - - - - - - STATES & TRANSITIONS - - - - - - - - - - """
    on = crest.State()
    off = current = crest.State()
    error = crest.State()
    
    off_to_on = crest.Transition(source=off, target=on, guard=(lambda self: self.switch_in.value == "on" and self.electricity_in.value >= 100))
    on_to_off = crest.Transition(source=on, target=off, guard=(lambda self: self.switch_in.value == "off" or self.electricity_in.value < 100))
    
    # transition to error state if the lamp ran for more than 1000.5 time units
    @crest.transition(source=on, target=error)
    def to_error(self):
        """More complex transitions can be defined as a function. We can use variables and calculations"""
        timeout = self.on_time.value >= 1000.5
        heat_is_on = self.heatelement.switch_in.value == "on"
        return timeout and heat_is_on
    
    """ - - - - - - - UPDATES - - - - - - - - - - """
    # LAMP is OFF or ERROR
    @crest.update(state=[off, error], target=lightelement.electricity_in)
    def update_light_elec_off(self, dt):
        # no electricity
        return 0

    @crest.update(state=[off, error], target=heatelement.electricity_in)
    def update_heat_elec_off(self, dt):
        # no electricity
        return 0
    
    
    
    # LAMP is ON
    @crest.update(state=on, target=lightelement.electricity_in)
    def update_light_elec_on(self, dt):
        # the lightelement gets the first 100Watt
        return 100
    
    @crest.update(state=on, target=heatelement.electricity_in)
    def update_heat_elec_on(self, dt):
        # the heatelement gets the rest
        return self.electricity_in.value - 100
        
    @crest.update(state=on, target=on_time)
    def update_time(self, dt):
        # also update the on_time so we know whether we overheat
        return self.on_time.value + dt
        
    """ - - - - - - - ACTIONS - - - - - - - - - - """
    # let's add an action that counts the number of times we switch to state "on"
    @crest.action(transition=off_to_on, target=on_count)
    def count_switching_on(self):
        """
        Actions are functions that are executed when the related transition is fired.
        Note that actions do not have a dt.
        """
        return self.on_count.value + 1

# create an instance!
elk.plot(GrowLamp())

# Simulation

Simulation allows us to execute the model and see its evolution.
`crestdsl`'s simulator is located in the `simultion` module. 
In order to use it, we have to import it.

In [ ]:
# import the simulator
from crestdsl.simulation import Simulator

After the import, we can use a simulator by initialising it with a system model.
In our case, we will explore the `GrowLamp` system that we defined above.

In [ ]:
gl = GrowLamp()
sim = Simulator(gl)

## Stabilisation
The simulator will execute the system's transitions, updates and influences until reaching a fixpoint.
This process is referred to as *stabilisation*.
Once stable, there are no more transitions can be triggered and all updates/influences/actions have been executed. 
After stabilisation, all ports have their correct values, calculted from preceeding ports.

In the GrowLamp, we see that the value's of the `temperature_out` and `light_out` ports are wrong (based on the dummy values we defined as their initial values).
After triggering the stabilisation, these values have been corrected.


The simulator also has a convenience API `plot()` that allows the direct plotting of the entity, without having to import and call the `elk` library.

In [ ]:
sim.stabilise()
sim.plot()

Stabilisaiton also has to be called after the modification of input values, such that the new values are used to update any dependent ports.
Further, all transitions have to be checked on whether they are enabled and executed if they are.

Below, we show the modification of the growlamp and stabilisation.
Compare the plot below to the plot above to see that the information has been updated.

In [ ]:
# modify the growlamp instance's inputs directly, the simulator points to that object and will use it
gl.electricity_in.value = 500
gl.switch_in.value = "on"
sim.stabilise()
sim.plot()

## Time advance
Evidently, we also want to simulate the behaviour over time.
The simulator's `advance(dt)` method does precisely that, by advancing `dt` time units.

Below we advance 500 time steps. 
The effect is that the global system time is now `t=500` (see the growing lamp's title bar).
Additionally, the local variable `on_time`, which sums up the total amount of time the automaton has spent in the `on` state, has the value of 500 too - Just as expected!

In [ ]:
sim.advance(500)
sim.plot()

# Where to go from here?

By now, you have seen how CREST and `crestdsl` can be used to define hybrid system models that combine discrete, autommata aspects with continuous time evolution.

`crestdsl` offers more functionality, including the formal verification through *timed CTL* model checking and the generation of system controllers.
To learn more about `crestdsl` go ahead and visit the source [repository](https://github.com/stklik/CREST/).